In [ ]:
from itertools import chain
import random

def generate_cb(i, valid = True):
    import json
    o = {
        "_label_cost": - (i % 2),
        "_label_probability": 0.5,
        "_label_Action": i % 2 + 1,
        "_labelIndex": i % 2,
        "a": [(i % 2) + 1, ((i + 1) % 2) + 1],
        "c": {'shared': {'f':'1'}, '_multi': [{'a': {'f': '1'}}, {'a': {'f': '2'}}]},
        "p": [0.5, 0.5] 
    }
    return f'{json.dumps(o, separators=(",", ":"))}{"" if valid else "!"}\n'


def generate_cb_rnd(i, nactions, epsilon):
    import json
    reward = random.random() < 1 - epsilon if i % nactions == 0 else random.random() < epsilon
    o = {
        "_label_cost": -reward,
        "_label_probability": 1 / nactions,
        "_label_Action": (i % nactions) + 1,
        "_labelIndex": i % nactions,
        "a": [(i % nactions) + 1] + [a + 1 for a in set(range(nactions)) - set([i % nactions])],
        "c": {'shared': {'f':'1'}, '_multi': [{'a': {'f': str(i)}} for i in range(nactions)]},
        "p": [1.0 / nactions]  * nactions
    }
    return f'{json.dumps(o, separators=(",", ":"))}\n'

def generate_file(generator, lines, path):
    with open(path, 'w') as f:
        f.writelines(map(lambda i: generator(i), range(lines)))

def generate_file_with_invalid(lines, path):
    with open(path, 'w') as f:
        for i in range(lines):
            f.write(generate_cb(i))
        f.write(generate_cb(0, False))
        for i in range(lines):
            f.write(generate_cb(i))          


In [ ]:
generate_file(generate_cb, 10, 'vw_executor/tests/data/cb_0.json')
generate_file(generate_cb, 11, 'vw_executor/tests/data/cb_1.json')
generate_file_with_invalid(10, 'vw_executor/tests/data/cb_invalid.json')

generate_file(lambda i: generate_cb_rnd(i, 8, 0.3), 100, 'vw_executor/tests/data/cb_100_0.json')
generate_file(lambda i: generate_cb_rnd(i, 8, 0.3), 100, 'vw_executor/tests/data/cb_100_1.json')